In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

import os

import numpy as np
import pandas as pd
import seaborn as sns

from app import app
from app.Investigation.DataInteractor.data_fetcher import DataFetcher

from heatmap import corrplot

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


import matplotlib.pyplot as plt

from scipy import integrate, stats
from app import app

# Regression models
import statsmodels.api as sm
from statsmodels.formula.api import ols
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

# Save model
import pickle

import os

from app.Investigation.Route_segmentation.segmentation import gen_traces

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
print(os.environ["PATH"])

import app.Investigation.OpenStreetMaps.associate_edges_to_operation as associate
# The OSM Graph


ModuleNotFoundError: No module named 'app.Investigation'; 'app' is not a package

# Read operation data

In [ ]:
name = "renault"  # input()
data_path = os.path.join(app.root_path) + "/DataBackup/" + name

try:
    loaded_data = pd.read_hdf(
        data_path + "_data.h5", key=name + "_updated_df_operation"
    )
    features = pd.read_hdf(data_path + "_data.h5", key=name + "_segments")
    features = segments[segments["mean_speed"] > 0]
    features = segments[segments["consumption_per_km"] < 10]
    features = segments[segments["consumption_per_km"] > -10]

except FileNotFoundError:
    print("Enter query")
    query = input() or "SELECT * from operation limit 10"
    print(query)
    data_fetcher = DataFetcher()
    data_fetcher.upload_data_to_h5(name, query)
    loaded_data = pd.read_hdf(
        data_path + "_data.h5", key=name + "_updated_df_operation"
    )
    features = pd.read_hdf(data_path + "_data.h5", key=name + "_segments")

loaded_data.vehicle_id.unique()

# Validation of run metric

In [ ]:

print(loaded_data['run'][loaded_data['vehicle_id']=='FSV110'].sum()/1000)
print(loaded_data['run'][loaded_data['vehicle_id']=='GHW284'].sum()/1000)
print(loaded_data['run'][loaded_data['vehicle_id']=='EGZ112'].sum()/1000, "\n")


print(loaded_data['odometer'][loaded_data['vehicle_id']=='FSV110'].iloc[-1] - loaded_data['odometer'][loaded_data['vehicle_id']=='FSV110'].iloc[0])
print(loaded_data['odometer'][loaded_data['vehicle_id']=='GHW284'].iloc[-1] - loaded_data['odometer'][loaded_data['vehicle_id']=='GHW284'].iloc[0])


In [ ]:
loaded_data = loaded_data[loaded_data.power_kw != 0]

#### Corrigiendo usuarios

In [ ]:

loaded_data.user_name[(loaded_data.user_name == '1er_alquiler') &
                        (loaded_data.vehicle_id =='FSV110')] = 'Santiago_Echavarria_FSV110'

loaded_data.user_name[(loaded_data.user_name == '1er_alquiler') &
                        (loaded_data.vehicle_id =='GHW284')] = 'Juan_David_Mira_Alq2'

loaded_data.user_name[(loaded_data.user_name == 'Juan_David_Mira') &
                        (loaded_data.timestamp2 > 1.608e9)] = 'Juan_David_Mira_03'

loaded_data.user_name[(loaded_data.user_name == 'Juan_David_Mira')] = 'Juan_David_Mira_01'

loaded_data.user_name[(loaded_data.user_name == 'Juan_Mira')] = 'Juan_David_Mira_02'

loaded_data.user_name.unique()

#### Corrigiendo los pesos
https://verne.elpais.com/verne/2016/04/19/articulo/1461079768_768006.html 61kg para mujeres y 67kg para hombres

In [ ]:
# Para el ZOE y LEAF respectivamente 
loaded_data.mass[loaded_data.mass == 170] = 1502
loaded_data.mass[loaded_data.mass == 1528] = 1562
loaded_data.mass[loaded_data.mass == 1584] = 1564

# Con acompañante mujer
loaded_data['mass'] = np.where((
    (loaded_data['user_name'] == 'Juan_David_Ochoa') |
    (loaded_data['user_name'] == 'Ricardo_Mejia') |
    (loaded_data['user_name'] == 'Jose_Alejandro_Montoya') |
    (loaded_data['user_name'] == 'Ana_Cristina_G') | 
    (loaded_data['user_name'] == 'Esterban_Betancur') | 
    (loaded_data['user_name'] == 'Juan_Gregorio_Arrieta') |
    (loaded_data['user_name'] == 'Sergio')),loaded_data.mass + 61 + 67, loaded_data.mass)

# Con acompañante hombre
loaded_data['mass'] = np.where((
    (loaded_data['user_name'] == 'Juan_David_Mira_01')),
    loaded_data.mass + 67 + 67, loaded_data.mass)

# Los que manejaron solos (menos peso)
loaded_data['mass'] = np.where((
    (loaded_data['user_name'] == 'Santiago_Echavarria_FSV110') |
    (loaded_data['user_name'] == 'Santiago_Echavarria_01') |
    (loaded_data['user_name'] == 'Santiago_Echavarria') |
    (loaded_data['user_name'] == 'Juan_David_Mira_Alq2') |
    (loaded_data['user_name'] == 'Juan_David_Mira_02') |
    (loaded_data['user_name'] == 'Jesus_Villa')),
    loaded_data.mass + 62, loaded_data.mass)

# Otros que manejaron solos 
loaded_data['mass'] = np.where((
    (loaded_data['user_name'] == 'David_Rios') |
    (loaded_data['user_name'] == 'Mauricio_Fernandez')),
    loaded_data.mass + 67, loaded_data.mass)

In [ ]:
plt.figure(figsize=[12, 8])
plt.scatter(x=loaded_data.timestamp2, y=loaded_data.user_name)

# Segmentation

# Relacion entre pedal de aceleración y potencia


In [ ]:
FRV020 = loaded_data[
    (loaded_data['vehicle_id'] == 'FRV020') &
    ((loaded_data['power_kw'] < -1) | (loaded_data['power_kw'] > 1)) &
    (loaded_data['throttle'] > 1) &
    ((loaded_data['throttle'] < 29) | (loaded_data['throttle'] > 31))]
FRV020_esteban = FRV020[FRV020['user_name'] == 'Esteban_Betancur']
plt.figure()
plt.scatter(x=FRV020['power_kw'], y=FRV020['throttle'])

#Correlacion
corr_coef, p_value = stats.pearsonr(FRV020['power_kw'].to_numpy(), FRV020['throttle'])
print('Corr with power = ',corr_coef)
print('P value = ',p_value)

corr_coef_curr, p_value_curr = stats.pearsonr(FRV020['current'].to_numpy(), FRV020['throttle'])
print('Corr with current = ',corr_coef_curr)
print('P value = ',p_value_curr)

corr_coef_acc, p_value_acc = stats.pearsonr(FRV020['mean_acc'].to_numpy(), FRV020['throttle'])
print('Corr with acc = ',corr_coef_acc)
print('P value = ',p_value_acc)
print('len = ',len(FRV020))

In [ ]:
features


In [ ]:
from sklearn.preprocessing import LabelEncoder  
features = features[features['kms'] <= 1.6]
features = features[features['min_acc'] > -5]
# features = features[(features['max_current'] < 250) ]

# #features = features[(features['traffic_factor'] < 45) ]

# # Cuando es con 1200 mts
features = features[features['consumption'] < 1]

# # Cuando es con 300mts
features = features[(features['consumption'] > -0.5) & (features['consumption'] < 0.6)]

# #features = features[(features['consumption_per_km'] < 0.8) ]

# features = features[features['std_acc'] != 0]

# #Solamente Zoe
# features = features[(features['vehicle_id'] != 'FRV020') & (features['vehicle_id'] != 'FVQ731')]

# features = features[(features['consumption_per_km'] > -4) | (features['consumption_per_km'] < 1) ].dropna()


features.corr()['consumption'].sort_values()


In [ ]:
heat_cols = [
    'mean_acc', 'mean_temp',
    'skew_acc', 'slope',
    'mean_power','mean_current', 'std_power','std_current',
    'osm_speed', 'mean_soc',
    'consumption_per_km'
]

a = features.copy()
a = a.rename(columns={'mean_speed':'osm_speed'})
corr=a[heat_cols].corr()
plt.figure(figsize=(8,8))
corrplot(corr.sort_values(['slope']), size_scale=500);

In [ ]:
plt.scatter(x=features.consumption, y=features.slope)

In [ ]:
a = a[a['mean_power']!= 0]
sns.pairplot(a.dropna().sort_values(['slope']), hue='slope_cat', palette='mako', vars=[
#     'min_acc',
    'slope',
    'mean_power',
#     'osm_speed',
#     'consumption_per_km'
], kind= 'scatter', height=8)

In [ ]:
rpm = loaded_data[loaded_data.rpm > 0]
plt.scatter(x=rpm.rpm, y=rpm.speed)


# To implement - pace during route
### TellsF the user if his current rate is lower or upper to the estimated

In [ ]:
features

In [ ]:
x=features.copy() ##[(features['slope'] > -5) & (features['slope'] < 5)]
# x['slope_cat'] = pd.cut(x["slope"], np.arange(-5.1,5.2,3.4) )
x = x[x['consumption_per_km'] > -0.2]

x = x[x['user_name'] != 'Juan_David_Mira_01']
x = x[x['user_name'] != 'Santiago_Echavarria_FSV110']

x = x[x['user_name'] != 'Santiago_Echavarria']
x = x[x['user_name'] != 'Santiago_Echavarria_01']  # Por ser muy atipico
x = x[x['user_name'] != 'Juan_David_Mira_01']
x = x[x['user_name'] != 'Juan_David_Mira_02']
x = x[x['user_name'] != 'Juan_David_Mira_03']
x = x[x['user_name'] != 'Juan_David_Mira_Alq2']

x.loc[x['user_name'] == 'Ricardo_Mejia', 'user_name'] = 'Driver 0'
x.loc[x['user_name'] == 'Jose_Alejandro_Montoya', 'user_name'] = 'Driver 1'
x.loc[x['user_name'] == 'Juan_Gregorio_Arrieta', 'user_name'] = 'Driver 2'
x.loc[x['user_name'] == 'Mauricio_Fernandez', 'user_name'] = 'Driver 3'
x.loc[x['user_name'] == 'Sergio', 'user_name'] = 'Driver 4'
x.loc[x['user_name'] == 'Jesus_Villa', 'user_name'] = 'Driver 5'
x.loc[x['user_name'] == 'Juan_David_Ochoa', 'user_name'] = 'Driver 6'
x.loc[x['user_name'] == 'David_Rios', 'user_name'] = 'Driver 7'
x.loc[x['user_name'] == 'Ana_Cristina_G', 'user_name'] = 'Driver 8'
x.loc[x['user_name'] == 'Esterban_Betancur', 'user_name'] = 'Driver 9'
x.loc[x['user_name'] == 'usuarios_eafit_vehiculo_2', 'user_name'] = 'Driver 10'
x.loc[x['user_name'] == 'sechava4', 'user_name'] = 'Driver 11'


x = x[(x['user_name'] == 'Driver 10') | (x['user_name'] == 'Driver 11')]
x.rename(columns={"mean_power": "MBPO (kW)","mean_speed": "Nominal speed (km/h)",
                                       'consumption_per_km': 'Consumption rate (kWh/km)','slope_cat':'Slope category'}, inplace=True)


sns.catplot(x='Slope category', y='Consumption rate (kWh/km)', hue='user_name', data=x.sort_values(by=['user_name', 'slope' ]), kind='box')

In [ ]:
sns.catplot(x='Slope category', y="MBPO (kW)", hue='user_name', data=x.sort_values(by=['user_name', 'slope' ]), kind='box')

In [ ]:
sns.catplot(x='Slope category', y='Nominal speed (km/h)', hue='user_name', data=x.sort_values(by=['user_name', 'slope' ]), kind='box')

In [ ]:
from   scipy import stats

x1=x[(x['slope'] > 2) ]
x2=x[(x['slope'] > -2) & (x['slope'] <= 2)]

stat, p, med, tbl = stats.median_test(x1['Consumption rate (kWh/km)'][(x1['user_name'] == 'Driver 2')],
                                      x1['Consumption rate (kWh/km)'][(x1['user_name'] == 'Driver 6')])
print('p = ' + str(p), '\n')

# Different slope group
t2, p2 = stats.ttest_ind(x1['Consumption rate (kWh/km)'][(x1['user_name'] == 'Driver 2')] ,
                         x2['Consumption rate (kWh/km)'][x2['user_name'] == 'Driver 6' ])
print('For same user - different slope group')
print('t = ' + str(t2))
print('p = ' + str(p2), '\n')

# Same slope group > 1.7
t2, p2 = stats.ttest_ind(x1['Consumption rate (kWh/km)'][(x1['user_name'] == 'Driver 2')] ,
                         x1['Consumption rate (kWh/km)'][x1['user_name'] == 'Driver 6' ])
print('For 2 different users - slope group > 2')
print('t = ' + str(t2))
print('p = ' + str(p2), '\n')

# Same slope group > 1.7
t2, p2 = stats.ttest_ind(x1['Nominal speed (km/h)'][(x1['user_name'] == 'Driver 2')] ,
                         x1['Nominal speed (km/h)'][x1['user_name'] == 'Driver 6' ])
print('Nominal speed (km/h) For 2 different users - slope group > 2')
print('t = ' + str(t2))
print('p = ' + str(p2), '\n')

# Same slope group > 1.7
t2, p2 = stats.ttest_ind(x1['MBPO (kW)'][(x1['user_name'] == 'Driver 2')] ,
                         x1['MBPO (kW)'][x1['user_name'] == 'Driver 6' ])
print('MBPO (kW) For 2 different users - slope group > 2')
print('t = ' + str(t2))
print('p = ' + str(p2), '\n')

# Separate 25 km for measuring and rest for test (for the test cases)


In [ ]:
features['cumdist1'] = features[(features['user_name'] == 'Santiago_Echavarria') ].kms.cumsum()  #features['cumdist'] 
features.loc[features['cumdist1'] > 70, 'user_name'] = 'Santiago_Echavarria_test'
features.loc[features['cumdist1'] < 70, 'user_name'] = 'Santiago_Echavarria_measure'

### Se utiliza para train los datos de las pruebas contraladas de todos los usuariois

In [ ]:
train = features[(features['user_name'] != 'Santiago_Echavarria_test') &
                 (features['user_name'] != 'Santiago_Echavarria_measure')]
test = features.loc[features.index.difference(train.index)]


In [ ]:
train.user_name.unique()

# Slope groups

In [ ]:
slope_groups = train.groupby(by=["slope_cat"])

mean_features_by_slope = slope_groups[['mean_power', 'min_acc', 'consumption_per_km']].mean().reset_index()
mean_features_by_slope.rename(columns={
    "mean_power": "mean_power_by_slope", 
    "min_acc": "mean_min_acc",
    'consumption_per_km': 'mean_consumption_per_km',
    'slope':'slope_cat'
}, inplace=True)

mean_features_by_slope.to_csv('UserDrivingData/mean_features_by_slope.csv', index=False)


#Se lee la que se saco con segmentos de 300m.
mean_features_by_slope = pd.read_csv('UserDrivingData/mean_features_by_slope.csv')
mean_features_by_slope['slope_cat'] = mean_features_by_slope['slope_cat'].astype('string')


train_cons = pd.merge(left=train, right=mean_features_by_slope,
                      left_on='slope_cat', right_on='slope_cat')

train_cons['user_name'].unique()

# User and slope groups

In [ ]:
user_slope_groups = train.groupby(by=['slope_cat', 'user_name'])
mean_features_by_user_and_slope = user_slope_groups[['mean_power', 'min_acc', 'consumption_per_km']].mean().reset_index()
mean_features_by_user_and_slope.rename(columns={
    "mean_power": "mean_power_usr",
    "min_acc": "mean_min_acc_usr",
    'consumption_per_km': 'mean_consumption_per_km_usr'
}, inplace=True)

mean_features_by_user_and_slope
mean_features_by_user_and_slope.to_csv('UserDrivingData/mean_features_by_user_and_slope.csv', index=False)


#Se lee la que se saco con segmentos de 300m.
mean_features_by_user_and_slope = pd.read_csv('UserDrivingData/mean_features_by_user_and_slope.csv')
mean_features_by_user_and_slope['slope_cat'] = mean_features_by_user_and_slope['slope_cat'].astype('string')

train_cons = pd.merge(left=train_cons, right=mean_features_by_user_and_slope,
                              left_on=['user_name', 'slope_cat'], right_on=['user_name', 'slope_cat'])

# Esta tabla se sacó con segmentos de 300m
mean_features_by_user_and_slope.tail(60)

In [ ]:
# cols = ['max_power','slope']
cols = ['mean_power_usr', 'mean_min_acc_usr', 'mean_speed', 'slope']
model_ft = train_cons[cols]
selected_ft = train_cons[['mean_power_usr', 'mean_min_acc_usr', 'mean_speed', 'slope', 'consumption_per_km']]

print(len(selected_ft))

selected_ft.corr()

# Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

# For other sklearn models differente than linear regression
scaler = MinMaxScaler()
scaler.fit(train_cons[selected_ft.columns])

# For linear regression
scaler_lm = MinMaxScaler()
scaler_lm.fit(train_cons[model_ft.columns])

train_scaled = pd.DataFrame(scaler.transform(train_cons[selected_ft.columns]), columns=selected_ft.columns)

train_lm_scaled = pd.DataFrame(scaler_lm.transform(train_cons[model_ft.columns]), columns=model_ft.columns)
train_lm_scaled['consumption_per_km'] = train_scaled['consumption_per_km']

# Linear Model

In [ ]:
formula = 'consumption_per_km ~ mean_power_usr + mean_min_acc_usr + mean_speed + slope -1' # 0.84
#formula = 'consumption_per_km ~ slope + max_power -1' # 0.84


lm_consumo = ols(formula = formula, data = train_cons[selected_ft.columns]).fit()
print(lm_consumo.summary())

In [ ]:
plt.figure(figsize =[20,20])
fig=sm.graphics.plot_partregress_grid(lm_consumo)


# Test case :

Aqui dividimos el set de validación en 2 partes: Santiago_Echavarria_measure y Santiago_Echavarria_test.
Santiago_Echavarria_measure contiene 25 km de manejo y se utilizará para determinar los promedios de las conductas de usuario. El conjunto Santiago_Echavarria_test contiene los siguientes kilómetros y será el conjunto donde se aplicará el algoritmo basado en los promedios calculados de Santiago_Echavarria_measure.


In [ ]:


test_test = test[test['user_name']=='Santiago_Echavarria_test']
test_measure = test[test['user_name']=='Santiago_Echavarria_measure']


print(len(test_test))
print(len(test_measure))


slope_user_groups = test_measure.groupby(by=['slope_cat', 'user_name'])
mean_features_by_user_and_slope = slope_user_groups[['mean_power', 'min_acc', 'consumption_per_km']].mean().reset_index()
mean_features_by_user_and_slope.rename(columns={
    "mean_power": "mean_power_usr",
    "min_acc": "mean_min_acc_usr",
    'consumption_per_km': 'mean_consumption_per_km_usr'
}, inplace=True)

mean_features_by_user_and_slope.user_name = 'Santiago_Echavarria_test'
test_test = pd.merge(how='left', left=test_test, right=mean_features_by_user_and_slope,
                              left_on=['user_name', 'slope_cat'], right_on=['user_name', 'slope_cat'])
test_test

In [ ]:
test_test = pd.merge(how='left', left=test_test, right=mean_features_by_slope,
                     left_on=['slope_cat'], right_on=['slope_cat'])

test_test


In [ ]:
test_test[['user_name', 'slope_cat']] = test_test[['user_name', 'slope_cat']].astype('string')

In [ ]:
#mean_max_pot_per_user_and_slope['slope_cat'] = mean_max_pot_per_user_and_slope['slope_cat'].astype('string')

In [ ]:
# test_consolidated = pd.merge(how='left', left=test_consolidated, right=mean_max_pot_per_user_and_slope,
#                               left_on=['slope_cat', 'user_name'], right_on=['slope_cat', 'user_name'])
# test_consolidated

In [ ]:
test_test['mean_power_usr'] = test_test.apply(
    lambda row: row['mean_power_by_slope'] if np.isnan(row['mean_power_usr']) else row['mean_power_usr'],
    axis=1
)
test_test['mean_min_acc_usr'] = test_test.apply(
    lambda row: row['mean_min_acc'] if np.isnan(row['mean_min_acc_usr']) else row['mean_min_acc_usr'],
    axis=1
)
test_test['mean_consumption_per_km_usr'] = test_test.apply(
    lambda row: row['mean_consumption_per_km'] if np.isnan(row['mean_consumption_per_km_usr']) else row['mean_consumption_per_km_usr'],
    axis=1
)
test_test.isna().sum()

In [ ]:
print(test_test[cols].isna().sum() )

test_scaled = pd.DataFrame(
    scaler_lm.transform(
        test_test[model_ft.columns]
    ), columns=model_ft.columns)

test_scaled[cols].isna().sum()


In [ ]:
y_test = pd.DataFrame(
    scaler.transform(
        test_test[selected_ft.columns]
    ), columns=selected_ft.columns
)['consumption_per_km']

predictions = lm_consumo.predict(test_scaled)



### Without scaling

In [ ]:
test_test['consumption_per_km'] = test_test['consumption_per_km']
predictions = lm_consumo.predict(test_test[cols])
y_test_lm = test_test['consumption_per_km']

In [ ]:
dump(lm_consumo, open('UserDrivingData/linear_model.pkl', 'wb'))

In [ ]:
def model_evaluation(y_test, predictions):
    RMSE = np.sqrt(mean_squared_error(y_test, predictions))
    print('RMSE =',RMSE)
    
    rmspe = (np.sqrt(np.mean(np.square((y_test - predictions) / y_test)))) * 100
    print('RMSPE =',rmspe)
    
    rme = np.mean(np.abs((y_test - predictions) / y_test)) * 100
    print('RME =',rme)


    print('R2 =',r2_score(y_test, predictions))
    print('max error=',max(abs(y_test - predictions)))
    plt.figure()
    plt.scatter(x=y_test, y=predictions)

model_evaluation(y_test_lm, predictions)

# Random forest

In [ ]:
X = train_lm_scaled[cols].values
y = train_lm_scaled['consumption_per_km'].values

# Step 1
# n_estimators=[,,,,,], max_depth=[,,,,,,]

# Curva de aprendizaje
# Variar train size 

# Curva de complejidad
# x cada uno de los parámetros max_depth': 110, 'max_samples': 0.2, 'n_estimators': 45}
randF_regr = RandomForestRegressor(n_estimators=149, max_depth=5,
                                   random_state=0,max_features='auto' , criterion='mse', max_samples = 0.219)

randF_regr.fit(X, y)
y_pred = randF_regr.predict(test_scaled[cols].values)

model_evaluation(y_test, y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
print("Initialize Grid Search")
param_grid = { 
    'n_estimators': np.arange(5,70,5),
    'max_depth' : np.arange(2,20,2),
    'max_samples' : np.arange(0.1,0.9,0.1)
}
# create and fit a regression model, testing each parameter value

 

model = RandomForestRegressor()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_params_)

# Complexity curves

In [ ]:
from sklearn.metrics import mean_absolute_error

n_estimators = np.arange(1,200,1)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for estimators in n_estimators:
    reg = RandomForestRegressor(n_estimators = estimators, max_features='auto', max_depth=5, max_samples = 0.219,
                                criterion = 'mse', random_state = 0)
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(n_estimators, train_acc, 'b', label = 'Training RMSE')
plt.plot(n_estimators, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('number of estimators ', fontsize = 14)
# plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
n_estimators[val_acc.index(min(val_acc))]


In [ ]:
depths = np.arange(1,20,1)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for depth in depths:
    reg = RandomForestRegressor(n_estimators = 48, max_features='auto', max_samples = 0.219,
                                max_depth=depth, criterion = 'mse', random_state = 0, n_jobs=-1)
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(depths, train_acc, 'b', label = 'Training RMSE')
plt.plot(depths, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('Maximun tree depth', fontsize = 14)
# plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
depths[val_acc.index(min(val_acc))]


In [ ]:
max_samples = np.arange(0.1,0.9,0.01)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for max_sample in max_samples:
    reg = RandomForestRegressor(n_estimators = 48, max_features='auto', max_samples = max_sample,
                                max_depth=5, criterion = 'mse', random_state = 0, n_jobs=-1)
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(max_samples, train_acc, 'b', label = 'Training RMSE')
plt.plot(max_samples, val_acc, 'r', label = 'Validation RMSE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('Maximun samples', fontsize = 14)
# plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
max_samples[val_acc.index(min(val_acc))]

In [ ]:
features=train_lm_scaled.columns[[0, 1, 2, 3]]
importances = randF_regr.feature_importances_
indices = np.argsort(importances)

plt.figure(1)
#plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')

In [ ]:
dump(randF_regr, open('MachineLearningModels/randomForest_0_3_mean_consumption_maxerr_model.pkl', 'wb'))
dump(scaler, open('MachineLearningModels/scaler.pkl', 'wb'))
dump(scaler_lm, open('MachineLearningModels/scaler_lm.pkl', 'wb'))

# XGBoost

In [ ]:
## 
data_dmatrix = xgb.DMatrix(data=X,label=y)

# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.5,
#                 max_depth = 10, n_estimators = 5)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.7, learning_rate = 0.3,
                max_depth = 2, n_estimators = 41)


xg_reg.fit(X, y)

y_pred_xgb = xg_reg.predict(test_scaled[cols].values)
model_evaluation(y_test, y_pred_xgb)

# Save the model
dump(xg_reg, open('MachineLearningModels/xg_reg_model.pickle.dat', 'wb'))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
print("Initialize Grid Search")
param_distributions = { 
    'colsample_bytree' : np.arange(0.1, 1, 0.1),
    'learning_rate' : np.arange(0.1, 1, 0.1),
    'n_estimators': [2,3,4,5],
    'max_depth' : np.arange(10, 30, 2),
    'objective' :['reg:squarederror']
}
# create and fit a regression model, testing each parameter value

 
#     'n_estimators': np.arange(3,60,1),
#     'max_depth' : np.arange(3,40,1),
model = xgb.XGBRegressor()
grid = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=-1)
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
n_estimators = np.arange(2,50,1)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for estimators in n_estimators:
    
    reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.75, learning_rate = 0.3,
                max_depth = 2, n_estimators = estimators, n_jobs=-1)

    
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(n_estimators, train_acc, 'b', label = 'Training RMSE')
plt.plot(n_estimators, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('number of estimators ', fontsize = 14)
#plt.title('Complexity curves for xgboost', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
# El valor óptimo
n_estimators[val_acc.index(min(val_acc))]

In [ ]:
depths = np.arange(1,25,1)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for depth in depths:
    
    reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.75, learning_rate = 0.3,
                max_depth = depth, n_estimators = 19, n_jobs=-1) 
    
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(depths, train_acc, 'b', label = 'Training RMSE')
plt.plot(depths, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('Maximun tree depth', fontsize = 14)
#plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
depths[val_acc.index(min(val_acc))]

#### Percentage of columns samples by tree

In [ ]:
colsamples_bytree = np.arange(0.05,1,0.05)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for colsample_bytree in colsamples_bytree:
    
    reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = colsample_bytree, learning_rate = 0.3,
                max_depth = 2, n_estimators = 19, n_jobs=-1) 
    
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(colsamples_bytree, train_acc, 'b', label = 'Training RMSE')
plt.plot(colsamples_bytree, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('Column samples by tree', fontsize = 14)
#plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
colsamples_bytree[val_acc.index(min(val_acc))]

In [ ]:
learning_rates = np.arange(0.05,1,0.05)
X_test = test_scaled[cols].values

train_reg_results = []
validation_reg_results = []
train_reg_r2 = []
validation_reg_r2 = []

train_acc = []
val_acc = []
val_r2 = []
train_r2 = []

for learning_rate in learning_rates:
    
    reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = learning_rate,
                max_depth = 2, n_estimators = 41, n_jobs=-1) 
    
    reg.fit(X, y)
    # Predicting the Traint set results
    y_pred_train = reg.predict(X)
    train_reg_results.append(np.sqrt(mean_squared_error(y, y_pred_train)))

    y_pred = reg.predict(X_test)
    validation_reg_results.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    train_reg_r2.append(mean_absolute_error(y, y_pred_train))

    validation_reg_r2.append(mean_absolute_error(y_test, y_pred))

    #print('R2 =',r2_score(y_test, predictions))

for i in range(len(train_reg_results)):
    train_acc.append(train_reg_results[i])
    val_acc.append(validation_reg_results[i])
    train_r2.append(train_reg_r2[i])
    val_r2.append(validation_reg_r2[i])


plt.figure('Complexity curve')
plt.style.use('seaborn')
plt.plot(learning_rates, train_acc, 'b', label = 'Training RMSE')
plt.plot(learning_rates, val_acc, 'r', label = 'Validation RMSE')
# plt.plot(n_estimators, train_r2, 'c', label = 'Training MAE')
# plt.plot(n_estimators, val_r2, 'y', label = 'Validation MAE')
plt.ylabel('RMSE', fontsize = 14)
plt.xlabel('Learning rates', fontsize = 14)
#plt.title('Complexity curves for random forest', fontsize = 18, y = 1.03)
plt.legend()
plt.show()

In [ ]:
learning_rates[val_acc.index(min(val_acc))]

In [ ]:
ax = xgb.plot_tree(xg_reg,num_trees=17)
fig = ax.figure
fig.set_size_inches(30, 35)
# fig = plot.get_figure()
# fig.savefig('test2png.png')

In [ ]:
ax = xgb.plot_importance(xg_reg)
fig = ax.figure
fig.set_size_inches(5, 5)

In [ ]:
params = {"objective":"reg:squarederror",'colsample_bytree': 1,'learning_rate': 0.1,
                'max_depth': 2, 'n_estimators': 24}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results

# sklearn linear model

In [ ]:
X = train_lm_scaled[cols].values
y = train_lm_scaled['consumption_per_km'].values

from sklearn import linear_model

linear_regr = linear_model.LinearRegression(n_jobs=-1)

linear_regr.fit(X, y)
y_pred_linear = linear_regr.predict(test_scaled[cols].values)
model_evaluation(y_test, y_pred_linear)
dump(linear_regr, open('MachineLearningModels/linear_regr_sklearn.pkl', 'wb'))

# Ann

In [ ]:
X = train_lm_scaled[cols].values
y = train_lm_scaled['consumption_per_km'].values

from sklearn.neural_network import MLPRegressor

ann_regr = MLPRegressor( activation='relu', alpha=0, hidden_layer_sizes= (50), max_iter=705000,
                        learning_rate='adaptive', warm_start=True, tol=1e-4, solver='lbfgs').fit(X, y)

#regr = MLPRegressor(random_state=1, max_iter=500, solver='lbfgs', activation='relu', alpha=0.002, hidden_layer_sizes=(150,)).fit(X, y)
y_ann=ann_regr.predict(test_scaled[cols].values)
ann_regr.score(test_scaled[cols].values, y_test)

model_evaluation(y_test, y_ann)
dump(ann_regr, open('MachineLearningModels/ann_regr.pkl', 'wb'))

In [ ]:
print("Initialize Grid Search")
param_grid = { 
    'solver':['lbfgs'],
     #arning_rate': ["constant", "invscaling", "adaptive"],
    'hidden_layer_sizes': [(150,100), (150,)],
    'activation': ["logistic"],
    'max_iter':[10500],
    }


# create and fit a regression model, testing each parameter value


model = MLPRegressor()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_params_)

# Learning curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
 
    if axes is None:
        _, axes = plt.subplots(1, 2, figsize=(20, 5))

    axes[0].set_title(title)
    #if ylim is not None:
        #axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    print('max train = ', np.mean(train_scores))
    print('max test = ', np.mean(test_scores))
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)
    print('Fitting time = ', np.mean(fit_times))

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    '''
    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, np.sort(test_scores_mean), 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")
    '''
    return plt


title = "Learning Curves (Neural Network)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = MLPRegressor(random_state=1, activation='tanh', alpha=0,
                         hidden_layer_sizes= (110 ), max_iter=70500, solver='lbfgs', learning_rate='invscaling')
plot_learning_curve(estimator, title, X, y, # ylim=(0.7, 1.01),
                    cv=cv, n_jobs=-1)
plt.show()

plt.figure()
title = "Learning Curves (Random Forest)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = RandomForestRegressor(n_estimators = 53, max_features='auto', max_samples = 0.1,
                                  max_depth=8, criterion = 'mse', random_state = 0, n_jobs=-1)
plot_learning_curve(estimator, title, X, y, # ylim=(0.7, 1.01),
                    cv=cv, n_jobs=-1)

plt.show() 

plt.figure()
title = "Learning Curves (Ordinary least squares Linear Regression)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = linear_model.LinearRegression(n_jobs=-1)
plot_learning_curve(estimator, title, X, y,  # ylim=(0.7, 1.01),
                    cv=cv, n_jobs=-1)

plt.show()


plt.figure()
title = "Learning Curves (XGBoost)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.15,
                             max_depth = 2, n_estimators = 18)

plot_learning_curve(estimator, title, X, y, # ylim=(0.7, 1.01),
                    cv=cv, n_jobs=-1)

plt.show()